<a href="https://colab.research.google.com/github/paraery/parsingResume/blob/main/transformersModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
NER = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is yugesh and I live in India"

NameError: ignored

In [ ]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/textfortraining7.csv')

In [ ]:
lstTxt = df['text'].values.tolist()

#train

In [ ]:
pip install -q datasets transformers

     |████████████████████████████████| 451 kB 5.0 MB/s 
     |████████████████████████████████| 115 kB 57.6 MB/s 
     |████████████████████████████████| 212 kB 32.0 MB/s 
     |████████████████████████████████| 127 kB 66.0 MB/s 


In [ ]:
import re
def get_tokens_with_entities(raw_text: str):
    # split the text by spaces only if the space does not occur between square brackets
    # we do not want to split "multi-word" entity value yet
    raw_tokens = re.split(r"\s(?![^\[]*\])", raw_text)

    # a regex for matching the annotation according to our notation [entity_value](entity_name)
    entity_value_pattern = r"\[(?P<value>.+?)\]\((?P<entity>.+?)\)"
    entity_value_pattern_compiled = re.compile(entity_value_pattern, flags=re.I|re.M)

    tokens_with_entities = []

    for raw_token in raw_tokens:
        match = entity_value_pattern_compiled.match(raw_token)
        if match:
            raw_entity_name, raw_entity_value = match.group("entity"), match.group("value")

            # we prefix the name of entity differently
            # B- indicates beginning of an entity
            # I- indicates the token is not a new entity itself but rather a part of existing one
            for i, raw_entity_token in enumerate(re.split("\s", raw_entity_value)):
                entity_prefix = "B" if i == 0 else "I"
                entity_name = f"{entity_prefix}-{raw_entity_name}"
                tokens_with_entities.append((raw_entity_token, entity_name))
        else:
            tokens_with_entities.append((raw_token, "O"))

    return tokens_with_entities

In [ ]:
from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# note that I purposefully misspell Kathmandu to Kathamanduu
sample_input = "I come from Kathmanduu valley,(location) [Nepal](location)"
tokens, entities = list(zip(*get_tokens_with_entities(sample_input)))
tokenized_input = tokenizer(tokens, is_split_into_words=True)
print("Original tokens           : ", tokens)
print("After subword tokenization: ", tokenizer.convert_ids_to_tokens(tokenized_input['input_ids']))

NameError: ignored

In [ ]:
class NERDataMaker:
    def __init__(self, texts):
        self.unique_entities = []
        self.processed_texts = []

        temp_processed_texts = []
        for text in texts:
            tokens_with_entities = get_tokens_with_entities(text)
            for _, ent in tokens_with_entities:
                if ent not in self.unique_entities:
                    self.unique_entities.append(ent)
            temp_processed_texts.append(tokens_with_entities)

        self.unique_entities.sort(key=lambda ent: ent if ent != "O" else "")

        for tokens_with_entities in temp_processed_texts:
            self.processed_texts.append([(t, self.unique_entities.index(ent)) for t, ent in tokens_with_entities])

    @property
    def id2label(self):
        return dict(enumerate(self.unique_entities))

    @property
    def label2id(self):
        return {v:k for k, v in self.id2label.items()}

    def __len__(self):
        return len(self.processed_texts)

    def __getitem__(self, idx):
        def _process_tokens_for_one_text(id, tokens_with_encoded_entities):
            ner_tags = []
            tokens = []
            for t, ent in tokens_with_encoded_entities:
                ner_tags.append(ent)
                tokens.append(t)

            return {
                "id": id,
                "ner_tags": ner_tags,
                "tokens": tokens
            }

        tokens_with_encoded_entities = self.processed_texts[idx]
        if isinstance(idx, int):
            return _process_tokens_for_one_text(idx, tokens_with_encoded_entities)
        else:
            return [_process_tokens_for_one_text(i+idx.start, tee) for i, tee in enumerate(tokens_with_encoded_entities)]

    def as_hf_dataset(self, tokenizer):
        from datasets import Dataset, Features, Value, ClassLabel, Sequence
        def tokenize_and_align_labels(examples):
            tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

            labels = []
            for i, label in enumerate(examples[f"ner_tags"]):
                word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
                previous_word_idx = None
                label_ids = []
                for word_idx in word_ids:  # Set the special tokens to -100.
                    if word_idx is None:
                        label_ids.append(-100)
                    elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                        label_ids.append(label[word_idx])
                    else:
                        label_ids.append(-100)
                    previous_word_idx = word_idx
                labels.append(label_ids)

            tokenized_inputs["labels"] = labels
            return tokenized_inputs

        ids, ner_tags, tokens = [], [], []
        for i, pt in enumerate(self.processed_texts):
            ids.append(i)
            pt_tokens,pt_tags = list(zip(*pt))
            ner_tags.append(pt_tags)
            tokens.append(pt_tokens)
        data = {
            "id": ids,
            "ner_tags": ner_tags,
            "tokens": tokens
        }
        features = Features({
            "tokens": Sequence(Value("string")),
            "ner_tags": Sequence(ClassLabel(names=dm.unique_entities)),
            "id": Value("int32")
        })
        ds = Dataset.from_dict(data, features)
        tokenized_ds = ds.map(tokenize_and_align_labels, batched=True)
        return tokenized_ds

In [ ]:
dm = NERDataMaker(lstTxt)
print(f"total examples = {len(dm)}")
print(dm[0:3])

total examples = 46
[{'id': 0, 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'tokens': ['kasetsart', 'university', 'bachelor', 'of', 'accountancy', 'program', '2018', '-', 'present,', '3.37', 'gpax', 'satriwitthaya', '2', 'high', 'school', 'english-mathematics', 'program', 'graduated', 'in', '2017,', '3.67', 'gpax', 'educational', 'history', 'g¼', '̄\x9d\x84|æ;7pªp\x8e\x84©\x84ªæ;n', '\x93££', '1\x84p\x80\x84¼à\x91\x93¹', '\x01\x80p¹æp{\x93£\x93æú', '¼\x93æ\x93|p£;s\x91\x93ª', '\x93ª\x8e', '', '̄££p{', '̄¼pæ\x93', '̄ª', '', '̄©©êª\x93|pæ\x93', '̄ª', 'skills', '&', 'abilities', '2019', 'mc;', '̄\x8d;"/', 'n;', '\x93¼àæ;\x11pæ\x84

In [ ]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
#tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
#model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(dm.unique_entities), id2label=dm.id2label, label2id=dm.label2id)

RuntimeError: ignored

In [ ]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("drive/MyDrive/PersonEntModel7")
model = AutoModelForTokenClassification.from_pretrained("drive/MyDrive/PersonEntModel7")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Configuration saved in drive/MyDrive/PersonEntModel3/config.json
Model weights saved in drive/MyDrive/PersonEntModel3/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/PersonEntModel3/tokenizer_config.json
Special tokens file saved in drive/MyDrive/PersonEntModel3/special_tokens_map.json


('drive/MyDrive/PersonEntModel3/tokenizer_config.json',
 'drive/MyDrive/PersonEntModel3/special_tokens_map.json',
 'drive/MyDrive/PersonEntModel3/vocab.txt',
 'drive/MyDrive/PersonEntModel3/added_tokens.json',
 'drive/MyDrive/PersonEntModel3/tokenizer.json')

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=40,
    weight_decay=0.01,
)

train_ds = dm.as_hf_dataset(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=train_ds, # eval on training set! ONLY for DEMO!!
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
model.save_pretrained("drive/MyDrive/PersonEntModel7")
tokenizer.save_pretrained("drive/MyDrive/PersonEntModel7")


  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 46
  Num Epochs = 40
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 120
  Number of trainable parameters = 66365187
The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags. If id, tokens, ner_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__ca

Epoch,Training Loss,Validation Loss
1,No log,0.000468
2,No log,0.000071
3,No log,0.000021
4,No log,0.000024
5,No log,0.000021
6,No log,0.000018
7,No log,0.000017
8,No log,0.000017
9,No log,0.000017
10,No log,0.000017


***** Running Evaluation *****
  Num examples = 46
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags. If id, tokens, ner_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 46
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags. If id, tokens, ner_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 46
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags. If id, tokens, ner_tags are no

('drive/MyDrive/PersonEntModel7/tokenizer_config.json',
 'drive/MyDrive/PersonEntModel7/special_tokens_map.json',
 'drive/MyDrive/PersonEntModel7/vocab.txt',
 'drive/MyDrive/PersonEntModel7/added_tokens.json',
 'drive/MyDrive/PersonEntModel7/tokenizer.json')

In [ ]:
import pandas as pd
dftest = pd.read_csv('textInResume.csv')

In [ ]:
import pandas as pd
dfName = pd.read_csv('nameinresume.csv')

In [ ]:
lstName = (dfName['firstname'].values+' '+dfName['lastname'].values).tolist()
lstName

['anwar rajawana',
 'apinop soisuwan',
 'apiwut kittiparikun',
 'arthit thetkham',
 'aussadach masun',
 'autsadang somboonphol',
 'chanon sattrupinat',
 'chayanit sripradit',
 'chayanon juntarapartsavorn',
 'chotitouch supalanunt',
 'ektanat pupat',
 'harit piyapornthana',
 'haruthai jankrajung',
 'ittichote sornmeethong',
 'jarrukorn pensalaphan',
 'jitawat chanpraneet',
 'kamolchai suebnipon',
 'kamolwan penpetch',
 'kanittha setthapitayakul',
 'kitisak thossaensin',
 'kittiphong chankong',
 'komson packdeearporn',
 'kritdanai peerapolchaikul',
 'krittapad harnchang',
 'naphattharachon pilikamin',
 'naruedom kiatikoon',
 'narurong saeheng',
 'natakit lalitputtichoke',
 'natanop pimonsathian',
 'nattacha kingbualuang',
 'nattachai noogure',
 'nattawoot denduangchai',
 'natthapat phusrisom',
 'navapuvadol uraikul',
 'navara sirijarusvong',
 'nisarat wisesbantao',
 'nonthawat aphiwong',
 'nutratanon  mahakhet',
 'nuttamol janmanee',
 'pakavich veeranarapanich',
 'pakpoom poodsud',
 'par

In [ ]:
lsttest = dftest['text'].values.tolist()

In [ ]:
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple") # pass device=0 if using gpu
i=0
num=0
for test in lsttest:
  ent = pipe(test)
  txtName = ''
  txt =''
  for val in ent:
    txtName = txtName+" | " + val['word'].replace('##','')
    txt = txt+val['word'].replace('##','')
  #print(i,txtName,'('+txt+')')
  for name in lstName:
    if hex(id(name))==hex(id(txt)):
       num=num+1 
       print(num)
    break
  i= i+1

In [ ]:
acc = (num/87)*100
print(num,acc)
num

0 0.0


0

In [ ]:
pipe("""Busaba Supasawat        profile a self-starter and quick learner with two-year experience   in financial industry. versatile  skill  sets including problem solving,  quantitative and analytical skills. seeking for career in financial industry where i can utilize and further  develop my  skills  and knowledge to   bring innovative solutions helping company to achieve the company goal.   contact phone: 0838523456  email: kanittha.se@hotmail.com  address: 118/208 condo vtara 36,  soi saen-sa-buy, rama iv rd.,  phra-khanong, klong-toei, bangkok  10110  work experience structured products development - bualuang securities may 2020  present ? develop and conduct structured product pricing and other relevant areas ? analyze data to efficiently expand client-based market ? provide market outlook and trading strategies of structured note to the sales team ? create new tools used to assist the sales team  ? handle project for improving current working flow  derivatives analyst - citibank n.a. april 2019  april 2020 ? review and improve the existing process to increase efficiency by applying automation process ? process settlement and confirmations via verbal and electronics means for derivative and structures product ? handle exceptions and providing constructive solutions to resolve issues  onsite analyst - central group online september 2018  march 2019 ? develop excel spreadsheet to improve day-to-day job ? perform ad hoc analysis to identify business issues and draw recommendation in marketing performance management  ? develop performance tracking on google tag manager on website education tsinghua university, beijing, china [m.sc.] aug 2016  july 2018  ? management science and engineering (mse) program ? department of industrial engineering chulalongkorn university [b. eng.] may 2012  may 2016 ? department of chemical engineering  ? awarded second-class honors computer skills ? vba in excel (advance), python (intermediate), sql (basic) ? excellent working knowledge for tableau, power bi certificate  ic complex 1 language skills  chinese (hsk5)  english (intermediate)   ----------------page (0) break---------------- """)

[{'entity_group': 'PERSON',
  'score': 0.9961846,
  'word': 'busaba supasawat',
  'start': 0,
  'end': 16}]

In [ ]:
from transformers import pipeline
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple") # pass device=0 if using gpu
pipe("""Busaba Supasawat        profile a self-starter and quick learner with two-year experience   in financial industry. versatile  skill  sets including problem solving,  quantitative and analytical skills. seeking for career in financial industry where i can utilize and further  develop my  skills  and knowledge to   bring innovative solutions helping company to achieve the company goal.   contact phone: 0838523456  email: kanittha.se@hotmail.com  address: 118/208 condo vtara 36,  soi saen-sa-buy, rama iv rd.,  phra-khanong, klong-toei, bangkok  10110  work experience structured products development - bualuang securities may 2020  present ? develop and conduct structured product pricing and other relevant areas ? analyze data to efficiently expand client-based market ? provide market outlook and trading strategies of structured note to the sales team ? create new tools used to assist the sales team  ? handle project for improving current working flow  derivatives analyst - citibank n.a. april 2019  april 2020 ? review and improve the existing process to increase efficiency by applying automation process ? process settlement and confirmations via verbal and electronics means for derivative and structures product ? handle exceptions and providing constructive solutions to resolve issues  onsite analyst - central group online september 2018  march 2019 ? develop excel spreadsheet to improve day-to-day job ? perform ad hoc analysis to identify business issues and draw recommendation in marketing performance management  ? develop performance tracking on google tag manager on website education tsinghua university, beijing, china [m.sc.] aug 2016  july 2018  ? management science and engineering (mse) program ? department of industrial engineering chulalongkorn university [b. eng.] may 2012  may 2016 ? department of chemical engineering  ? awarded second-class honors computer skills ? vba in excel (advance), python (intermediate), sql (basic) ? excellent working knowledge for tableau, power bi certificate  ic complex 1 language skills  chinese (hsk5)  english (intermediate)   ----------------page (0) break---------------- """)

[{'entity_group': 'person',
  'score': 0.360387,
  'word': 'bus',
  'start': 0,
  'end': 3},
 {'entity_group': 'person',
  'score': 0.5049531,
  'word': 'supasawa',
  'start': 7,
  'end': 15},
 {'entity_group': 'contact',
  'score': 0.7399174,
  'word': 'contact phone : 0838523456 email : kanittha.',
  'start': 389,
  'end': 432},
 {'entity_group': 'contact',
  'score': 0.4226862,
  'word': '.',
  'start': 442,
  'end': 443},
 {'entity_group': 'education',
  'score': 0.50174844,
  'word': '##ing',
  'start': 1616,
  'end': 1619},
 {'entity_group': 'education',
  'score': 0.90360725,
  'word': 'department of industrial engineering chulalongkorn university [ b. eng. ] may 2012 may 2016? department of chemical engineering? awarded',
  'start': 1733,
  'end': 1872},
 {'entity_group': 'education',
  'score': 0.66518116,
  'word': '- class',
  'start': 1879,
  'end': 1885}]